# GPU

In [9]:
!nvidia-smi

Mon Jun 17 11:17:21 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   53C    P8              5W /   40W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [12]:
torch.cuda.device_count()

1

In [13]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [14]:
x = torch.tensor([1, 2, 3])
x.device #默认创建的张量是在cpu上

device(type='cpu')

In [16]:
X = torch.ones(2, 3, device=try_gpu()) #在创建的时候选择在gpu上创建
X,X.device

(tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'),
 device(type='cuda', index=0))

In [18]:
Y = torch.rand(2, 3, device=try_gpu(1))#在第二块gpu上出创建 但这里没有第二块gpu
Y,Y.device

(tensor([[0.3518, 0.3702, 0.9791],
         [0.9148, 0.6418, 0.4834]]),
 device(type='cpu'))

In [21]:
#在第一块gpu上随机创建一个张量
Y = torch.rand(2, 3, device=try_gpu(0))
Y,Y.device

(tensor([[0.7126, 0.8746, 0.0561],
         [0.4553, 0.1403, 0.7474]], device='cuda:0'),
 device(type='cuda', index=0))

In [23]:
#要计算X+Y
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [24]:
Y + Z #Y和Z都在同一个GPU上

tensor([[1.7126, 1.8746, 1.0561],
        [1.4553, 1.1403, 1.7474]], device='cuda:0')

In [26]:
Z.cuda(0) is Z

True

In [27]:
# 代码将模型参数放在GPU上
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [28]:
net(X)

tensor([[-0.0765],
        [-0.0765]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [29]:
net[0].weight.data.device

device(type='cuda', index=0)